## etri 전처리 코드 제작

In [1]:
from pathlib import Path
import pickle

import os
import argparse
import itertools
from det3d.torchie import Config
# 주피터 노트북 경로설정
os.chdir('../')
def parse_args():
    parser = argparse.ArgumentParser(description="Train a detector")
    parser.add_argument("--config", default="configs/etriInfra/pp/etriInfra_centerpoint_pp_02voxel_two_pfn_10sweep.py", help="train config file path")
    parser.add_argument("--work_dir", help="the dir to save logs and models")
    parser.add_argument("--resume_from", help="the checkpoint file to resume from")
    parser.add_argument(
        "--validate",
        action="store_true",
        help="whether to evaluate the checkpoint during training",
    )
    parser.add_argument(
        "--gpus",
        type=int,
        default=1,
        help="number of gpus to use " "(only applicable to non-distributed training)",
    )
    parser.add_argument("--seed", type=int, default=None, help="random seed")
    parser.add_argument(
        "--launcher",
        choices=["pytorch", "slurm"],
        default="pytorch",
        help="job launcher",
    )
    parser.add_argument("--local_rank", type=int, default=0)
    parser.add_argument(
        "--autoscale-lr",
        action="store_true",
        help="automatically scale lr with the number of gpus",
    )
    args = parser.parse_args(args=[])
    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(args.local_rank)

    return args
args = parse_args()
cfg = Config.fromfile(args.config)

In [2]:
# 로딩에 약 4분 소요
data_path = "/home/jaelee/objdect/CenterPoint/data/etri3Dobj_infra_edge/3D_object_detection_tracking(edge_infra)/infos_train.pkl"
with open(data_path, "rb") as f:
    etri_data_all = pickle.load(f)

In [3]:
_num_point_features = 3
test_mode = False

res = {
    "lidar": {
        "type": "lidar",
        "points": None,
        "nsweeps": 1,
        # "ground_plane": -gp[-1] if with_gp else None,
        "annotations": None,
    },
    "metadata": {
        "image_prefix": Path("data/etri3Dobj_infra_edge"),
        "num_point_features": _num_point_features,
    },
    "calib": None,
    "cam": {},
    "mode": "val" if test_mode else "train"
}

In [4]:
len(etri_data_all)

25280

In [5]:
frame_data = etri_data_all[0]
# frame_data

In [6]:
# LoadPointCloudFromFile
def mResPoint(res, frame_data):
    res["lidar"]["points"] = frame_data["points"]
    res["type"] = "etrInfraDataset"
mResPoint(res, frame_data)
# res

In [7]:
# LoadPointCloudAnnotations
def mResAnno(res, frame_data):
    res["lidar"]["annotations"] = frame_data["annotations"]
mResAnno(res, frame_data)
# res

In [8]:
# Preprocess
from det3d.builder import build_dbsampler
import numpy as np
from det3d.core.bbox import box_np_ops
from det3d.core.sampler import preprocess as prep

def _dict_select(dict_, inds):
    for k, v in dict_.items():
        if isinstance(v, dict):
            _dict_select(v, inds)
        else:
            dict_[k] = v[inds]

def drop_arrays_by_name(gt_names, used_classes):
    inds = [i for i, x in enumerate(gt_names) if x not in used_classes]
    inds = np.array(inds, dtype=np.int64)
    return inds

info_path = "data/etri3Dobj_infra_edge/db_meta_withoutvelo.pkl"
def mPreprocess(res, frame_data, cfg):
    args.config = "configs/etriInfra/pp/etriInfra_centerpoint_pp_02voxel_two_pfn_10sweep.py"
    
    class_names = list(itertools.chain(*[t["class_names"] for t in cfg.tasks]))
    train_preprocessor = dict(
        mode="train",
        shuffle_points=True,
        global_rot_noise=[-0.3925, 0.3925],
        global_scale_noise=[0.95, 1.05],
        db_sampler=cfg.db_sampler,
        class_names=class_names,
        min_points_in_gt = 5,
    )
    shuffle_points = train_preprocessor["shuffle_points"]
    min_points_in_gt = train_preprocessor.get("min_points_in_gt", -1)
    mode = train_preprocessor["mode"]
    no_augmentation = cfg.get('no_augmentation', False)
    info_path = cfg.db_sampler.db_info_path
    
    if mode == "train":
        global_rotation_noise = train_preprocessor["global_rot_noise"]
        global_scaling_noise = train_preprocessor["global_scale_noise"]
        global_translate_std = train_preprocessor.get('global_translate_std', 0)
        class_names = train_preprocessor["class_names"]
    if mode == "train":
        if train_preprocessor["db_sampler"] != None:
            db_sampler = build_dbsampler(train_preprocessor["db_sampler"])
        else:
            db_sampler = None 
            
    # with open(info_path, "rb") as f:
    #     db_infos = pickle.load(f)
    res["mode"] = mode
    points = res["lidar"]["points"]
    if mode == "train":
        anno_dict = res["lidar"]["annotations"]
        
        gt_dict = {
            "gt_boxes": anno_dict["boxes"],
            "gt_names": np.array(anno_dict["names"]).reshape(-1),
        }
        
    if mode == "train" and not no_augmentation:
        selected = drop_arrays_by_name(
                    gt_dict["gt_names"], ["DontCare", "ignore", "UNKNOWN"]
                )
        _dict_select(gt_dict, selected)
        
        if min_points_in_gt > 0:
            point_counts = box_np_ops.points_count_rbbox(
                points, gt_dict["gt_boxes"]
            )
            mask = point_counts >= min_points_in_gt
            _dict_select(gt_dict, mask)
        
        gt_boxes_mask = np.array(
                    [n in class_names for n in gt_dict["gt_names"]], dtype=np.bool_
                )
    if db_sampler:
        sampled_dict = db_sampler.sample_all(
            "",
            gt_dict["gt_boxes"],
            gt_dict["gt_names"],
            res["metadata"]["num_point_features"],
            False,
            gt_group_ids=None,
            calib=None,
            road_planes=None
        )

        if sampled_dict is not None:
            sampled_gt_names = sampled_dict["gt_names"]
            sampled_gt_boxes = sampled_dict["gt_boxes"]
            sampled_points = sampled_dict["points"]
            sampled_gt_masks = sampled_dict["gt_masks"]
            gt_dict["gt_names"] = np.concatenate(
                [gt_dict["gt_names"], sampled_gt_names], axis=0
            )
            gt_dict["gt_boxes"] = np.concatenate(
                [gt_dict["gt_boxes"], sampled_gt_boxes]
            )
            gt_boxes_mask = np.concatenate(
                [gt_boxes_mask, sampled_gt_masks], axis=0
            )


            points = np.concatenate([sampled_points, points], axis=0)

    _dict_select(gt_dict, gt_boxes_mask)

    gt_classes = np.array(
        [class_names.index(n) + 1 for n in gt_dict["gt_names"]],
        dtype=np.int32,
    )
    gt_dict["gt_classes"] = gt_classes

    gt_dict["gt_boxes"], points = prep.random_flip_both(gt_dict["gt_boxes"], points)

    gt_dict["gt_boxes"], points = prep.global_rotation(
        gt_dict["gt_boxes"], points, rotation=global_rotation_noise
    )
    gt_dict["gt_boxes"], points = prep.global_scaling_v2(
        gt_dict["gt_boxes"], points, *global_scaling_noise
    )
    gt_dict["gt_boxes"], points = prep.global_translate_(
        gt_dict["gt_boxes"], points, noise_translate_std=global_translate_std
    )
    if shuffle_points:
        np.random.shuffle(points)

    res["lidar"]["points"] = points

    if mode == "train":
        res["lidar"]["annotations"] = gt_dict
    return
mPreprocess(res, frame_data, cfg)
# res


/home/jaelee/objdect/CenterPoint/det3d/core/bbox/geometry.py:149: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/home/jaelee/objdect/CenterPoint/det3d/core/bbox/geometry.py:162: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/home/jaelee/objdect/CenterPoint/det3d/core/bbox/geometry.py:280: NumbaDeprecationWa

no apex


In [9]:
#Voxelization
from det3d.core.input.voxel_generator import VoxelGenerator
vxg = cfg.voxel_generator
ranger = vxg.range
voxel_size = vxg.voxel_size
max_points_in_voxel = vxg.max_points_in_voxel
max_voxel_num = [vxg.max_voxel_num, vxg.max_voxel_num] if isinstance(vxg.max_voxel_num, int) else vxg.max_voxel_num

double_flip = vxg.get('double_flip', False)

voxel_generator = VoxelGenerator(
    voxel_size=voxel_size,
    point_cloud_range=ranger,
    max_num_points=max_points_in_voxel,
    max_voxels=max_voxel_num[0])


In [10]:
voxel_size = voxel_generator.voxel_size
pc_range = voxel_generator.point_cloud_range
grid_size = voxel_generator.grid_size

In [11]:
if res["mode"] == "train":
    gt_dict = res["lidar"]["annotations"]
    bv_range = pc_range[[0, 1, 3, 4]]
    mask = prep.filter_gt_box_outside_range(gt_dict["gt_boxes"], bv_range)
    _dict_select(gt_dict, mask)

    res["lidar"]["annotations"] = gt_dict
    max_voxels = max_voxel_num[0]
else:
    max_voxels = max_voxel_num[1]

/home/jaelee/objdect/CenterPoint/det3d/core/bbox/geometry.py:279: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "points_in_convex_polygon_jit" failed type inference due to: No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(float32, 3d, C), Tuple(slice<a:b>, list(int64)<iv=None>, slice<a:b>))
 
There are 22 candidate implementations:
   - Of which 20 did not match due to:
   Overload of function 'getitem': File: <numerous>: Line N/A.
     With argument(s): '(array(float32, 3d, C), Tuple(slice<a:b>, list(int64)<iv=None>, slice<a:b>))':
    No match.
   - Of which 2 did not match due to:
   Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 209.
     With argument(s): '(array(float32, 3d, C), Tuple(slice<a:b>, list(int64)<iv=None>, slice<a:b>))':
    Rejected as the implementation raised a specific error:
      NumbaTypeError: Unsupported 

In [12]:
voxels, coordinates, num_points = voxel_generator.generate(
    res["lidar"]["points"], max_voxels=max_voxels 
)

In [13]:
num_voxels = np.array([voxels.shape[0]], dtype=np.int64)

res["lidar"]["voxels"] = dict(
    voxels=voxels,
    coordinates=coordinates,
    num_points=num_points,
    num_voxels=num_voxels,
    shape=grid_size,
    range=pc_range,
    size=voxel_size
)

In [14]:
res

{'lidar': {'type': 'lidar',
  'points': array([[ -5.73508612,   4.17013045,  -1.90411989],
         [ -6.81193661,   4.66280996,  -1.90944888],
         [  8.49280075, -50.27583465,  10.76253949],
         ...,
         [  5.97048998,  -5.29747305,  -0.14517664],
         [ 13.05656451,  -1.48573312,  -0.69696109],
         [  0.89737355,  -1.33217416,  -0.65877836]]),
  'nsweeps': 1,
  'annotations': {'gt_boxes': array([[  4.93672064,   3.68360819,  -1.04639713,   2.20052156,
             4.28517147,   1.49311213,   0.        ,   0.        ,
             5.75996836],
          [ -0.15865307,  11.34789882,  -0.85099579,   2.18612882,
             5.12348858,   1.84161136,   0.        ,   0.        ,
             5.58543544],
          [  3.19586692,  12.14403596,  -0.98550483,   2.04894715,
             4.86731969,   1.62057821,   0.        ,   0.        ,
             5.58543544],
          [  9.53592463,  14.84466422,  -1.09922195,   2.18092847,
             4.88676171,   1.45245743,

In [15]:
# AssignLabel
import logging
from det3d.utils.config_tool import get_downsample_factor
from det3d.core.utils.center_utils import (
    draw_umich_gaussian, gaussian_radius
)

def flatten(box):
    return np.concatenate(box, axis=0)

def merge_multi_group_label(gt_classes, num_classes_by_task): 
    num_task = len(gt_classes)
    flag = 0 

    for i in range(num_task):
        gt_classes[i] += flag 
        flag += num_classes_by_task[i]

    return flatten(gt_classes)

# class imbalence를 해결하기 위해 그룹 구성. class가 balence되어 있으면 한 그룹으로 합쳐도 됨
tasks = [
    dict(num_class=2, class_names=["car", "personal_mobility"]),
    dict(num_class=2, class_names=["truck", "construction_vehicle"]),
    dict(num_class=1, class_names=["bus"]),
    dict(num_class=1, class_names=["ground_animal"]),
    dict(num_class=2, class_names=["motorcycle", "bicycle"]),
    dict(num_class=1, class_names=["pedestrian"]),
]
# training and testing settings
target_assigner = dict(
    tasks=tasks,
)
model = dict(
    type="PointPillars",
    pretrained=None,
    reader=dict(
        type="PillarFeatureNet",
        num_filters=[64, 64],
        num_input_features=5,
        with_distance=False,
        voxel_size=(0.2, 0.2, 8),
        pc_range=(-51.2, -51.2, -5.0, 51.2, 51.2, 3.0),
    ),
    backbone=dict(type="PointPillarsScatter", ds_factor=1),
    neck=dict(
        type="RPN",
        layer_nums=[3, 5, 5],
        ds_layer_strides=[2, 2, 2],
        ds_num_filters=[64, 128, 256],
        us_layer_strides=[0.5, 1, 2],
        us_num_filters=[128, 128, 128],
        num_input_features=64,
        logger=logging.getLogger("RPN"),
    ),
    bbox_head=dict(
        # type='RPNHead',
        type="CenterHead",
        in_channels=sum([128, 128, 128]),
        tasks=tasks,
        dataset='etriInfra',
        weight=0.25,
        # code_weights is the weight for regression loss of 'reg', 'height', 'dim', etc.. it is not related to classes
        code_weights=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 0.2, 1.0, 1.0],
        common_heads={'reg': (2, 2), 'height': (1, 2), 'dim':(3, 2), 'rot':(2, 2)}, # (output_channel, num_conv)
    ),
)
#     bbox_head=dict(
#         # type='RPNHead',
#         type="CenterHead",
#         in_channels=sum([128, 128, 128]),
#         tasks=tasks,
#         dataset='nuscenes',
#         weight=0.25,
#         code_weights=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 0.2, 1.0, 1.0],
#         common_heads={'reg': (2, 2), 'height': (1, 2), 'dim':(3, 2), 'rot':(2, 2), 'vel': (2, 2)}, # (output_channel, num_conv)
#     ),
# )
assigner = dict(
    target_assigner=target_assigner,
    out_size_factor=get_downsample_factor(model),
    gaussian_overlap=0.1,
    max_objs=500,
    min_radius=2,
)
assigner = dict(
    target_assigner=target_assigner,
    out_size_factor=get_downsample_factor(model),
    gaussian_overlap=0.1,
    max_objs=500,
    min_radius=2,
)
train_cfg = dict(assigner=assigner)
assigner_cfg = train_cfg["assigner"]

out_size_factor = assigner_cfg["out_size_factor"]
tasks = assigner_cfg["target_assigner"]["tasks"]
gaussian_overlap = assigner_cfg["gaussian_overlap"]
_max_objs = assigner_cfg["max_objs"]
_min_radius = assigner_cfg["min_radius"]
cfg = assigner_cfg

# __call__
max_objs = _max_objs
class_names_by_task = [t["class_names"] for t in tasks]
num_classes_by_task = [t["num_class"] for t in tasks]

example = {}

if res["mode"] == "train":
    # Calculate output featuremap size
    if 'voxels' in res['lidar']:
        # Calculate output featuremap size
        grid_size = res["lidar"]["voxels"]["shape"] 
        pc_range = res["lidar"]["voxels"]["range"]
        voxel_size = res["lidar"]["voxels"]["size"]
        feature_map_size = grid_size[:2] // out_size_factor
    else:
        pc_range = np.array(cfg['pc_range'], dtype=np.float32)
        voxel_size = np.array(cfg['voxel_size'], dtype=np.float32)
        grid_size = (pc_range[3:] - pc_range[:3]) / voxel_size
        grid_size = np.round(grid_size).astype(np.int64)

    feature_map_size = grid_size[:2] // out_size_factor

    gt_dict = res["lidar"]["annotations"]

    # reorganize the gt_dict by tasks
    task_masks = []
    flag = 0
    for class_name in class_names_by_task:
        task_masks.append(
            [
                np.where(
                    gt_dict["gt_classes"] == class_name.index(i) + 1 + flag
                )
                for i in class_name
            ]
        )
        flag += len(class_name)

    task_boxes = []
    task_classes = []
    task_names = []
    flag2 = 0
    for idx, mask in enumerate(task_masks):
        task_box = []
        task_class = []
        task_name = []
        for m in mask:
            task_box.append(gt_dict["gt_boxes"][m])
            task_class.append(gt_dict["gt_classes"][m] - flag2)
            task_name.append(gt_dict["gt_names"][m])
        task_boxes.append(np.concatenate(task_box, axis=0))
        task_classes.append(np.concatenate(task_class))
        task_names.append(np.concatenate(task_name))
        flag2 += len(mask)

    for task_box in task_boxes:
        # limit rad to [-pi, pi]
        task_box[:, -1] = box_np_ops.limit_period(
            task_box[:, -1], offset=0.5, period=np.pi * 2
        )

    # print(gt_dict.keys())
    gt_dict["gt_classes"] = task_classes
    gt_dict["gt_names"] = task_names
    gt_dict["gt_boxes"] = task_boxes

    res["lidar"]["annotations"] = gt_dict

    draw_gaussian = draw_umich_gaussian

    hms, anno_boxs, inds, masks, cats = [], [], [], [], []

    for idx, task in enumerate(tasks):
        hm = np.zeros((len(class_names_by_task[idx]), feature_map_size[1], feature_map_size[0]),
                        dtype=np.float32)

        if res['type'] == 'NuScenesDataset':
            # [reg, hei, dim, vx, vy, rots, rotc]
            anno_box = np.zeros((max_objs, 10), dtype=np.float32)
        elif res['type'] == 'WaymoDataset':
            anno_box = np.zeros((max_objs, 10), dtype=np.float32) 
        else:
            anno_box = np.zeros((max_objs, 10), dtype=np.float32) 
            # raise NotImplementedError("Only Support nuScene for Now!")

        ind = np.zeros((max_objs), dtype=np.int64)
        mask = np.zeros((max_objs), dtype=np.uint8)
        cat = np.zeros((max_objs), dtype=np.int64)

        num_objs = min(gt_dict['gt_boxes'][idx].shape[0], max_objs)  

        for k in range(num_objs):
            cls_id = gt_dict['gt_classes'][idx][k] - 1

            w, l, h = gt_dict['gt_boxes'][idx][k][3], gt_dict['gt_boxes'][idx][k][4], \
                        gt_dict['gt_boxes'][idx][k][5]
            w, l = w / voxel_size[0] / out_size_factor, l / voxel_size[1] / out_size_factor
            if w > 0 and l > 0:
                radius = gaussian_radius((l, w), min_overlap=gaussian_overlap)
                radius = max(_min_radius, int(radius))

                # be really careful for the coordinate system of your box annotation. 
                x, y, z = gt_dict['gt_boxes'][idx][k][0], gt_dict['gt_boxes'][idx][k][1], \
                            gt_dict['gt_boxes'][idx][k][2]

                coor_x, coor_y = (x - pc_range[0]) / voxel_size[0] / out_size_factor, \
                                    (y - pc_range[1]) / voxel_size[1] / out_size_factor

                ct = np.array(
                    [coor_x, coor_y], dtype=np.float32)  
                ct_int = ct.astype(np.int32)

                # throw out not in range objects to avoid out of array area when creating the heatmap
                if not (0 <= ct_int[0] < feature_map_size[0] and 0 <= ct_int[1] < feature_map_size[1]):
                    continue 

                draw_gaussian(hm[cls_id], ct, radius)

                new_idx = k
                x, y = ct_int[0], ct_int[1]

                cat[new_idx] = cls_id
                ind[new_idx] = y * feature_map_size[0] + x
                mask[new_idx] = 1

                if res['type'] == 'NuScenesDataset' or res['type'] == 'etrInfraDataset': 
                    vx, vy = gt_dict['gt_boxes'][idx][k][6:8]
                    rot = gt_dict['gt_boxes'][idx][k][8]
                    anno_box[new_idx] = np.concatenate(
                        (ct - (x, y), z, np.log(gt_dict['gt_boxes'][idx][k][3:6]),
                        np.array(vx), np.array(vy), np.sin(rot), np.cos(rot)), axis=None)
                elif res['type'] == 'WaymoDataset':
                    vx, vy = gt_dict['gt_boxes'][idx][k][6:8]
                    rot = gt_dict['gt_boxes'][idx][k][-1]
                    anno_box[new_idx] = np.concatenate(
                    (ct - (x, y), z, np.log(gt_dict['gt_boxes'][idx][k][3:6]),
                    np.array(vx), np.array(vy), np.sin(rot), np.cos(rot)), axis=None)
                else:
                    raise NotImplementedError("Only Support Waymo and nuScene for Now")

        hms.append(hm)
        anno_boxs.append(anno_box)
        masks.append(mask)
        inds.append(ind)
        cats.append(cat)

    # used for two stage code 
    boxes = flatten(gt_dict['gt_boxes'])
    classes = merge_multi_group_label(gt_dict['gt_classes'], num_classes_by_task)

    if res["type"] == "NuScenesDataset" or res['type'] == 'etrInfraDataset':
        gt_boxes_and_cls = np.zeros((max_objs, 10), dtype=np.float32)
    elif res['type'] == "WaymoDataset":
        gt_boxes_and_cls = np.zeros((max_objs, 10), dtype=np.float32)
    else:
        raise NotImplementedError()

    boxes_and_cls = np.concatenate((boxes, 
        classes.reshape(-1, 1).astype(np.float32)), axis=1)
    num_obj = len(boxes_and_cls)
    assert num_obj <= max_objs
    # x, y, z, w, l, h, rotation_y, velocity_x, velocity_y, class_name
    boxes_and_cls = boxes_and_cls[:, [0, 1, 2, 3, 4, 5, 8, 6, 7, 9]]
    gt_boxes_and_cls[:num_obj] = boxes_and_cls

    example.update({'gt_boxes_and_cls': gt_boxes_and_cls})

    example.update({'hm': hms, 'anno_box': anno_boxs, 'ind': inds, 'mask': masks, 'cat': cats})
else:
    pass

res["lidar"]["targets"] = example

In [16]:
res

{'lidar': {'type': 'lidar',
  'points': array([[ -5.73508612,   4.17013045,  -1.90411989],
         [ -6.81193661,   4.66280996,  -1.90944888],
         [  8.49280075, -50.27583465,  10.76253949],
         ...,
         [  5.97048998,  -5.29747305,  -0.14517664],
         [ 13.05656451,  -1.48573312,  -0.69696109],
         [  0.89737355,  -1.33217416,  -0.65877836]]),
  'nsweeps': 1,
  'annotations': {'gt_boxes': [array([[  4.93672064,   3.68360819,  -1.04639713,   2.20052156,
              4.28517147,   1.49311213,   0.        ,   0.        ,
             -0.52321694],
           [ -0.15865307,  11.34789882,  -0.85099579,   2.18612882,
              5.12348858,   1.84161136,   0.        ,   0.        ,
             -0.69774987],
           [  3.19586692,  12.14403596,  -0.98550483,   2.04894715,
              4.86731969,   1.62057821,   0.        ,   0.        ,
             -0.69774987],
           [  9.53592463,  14.84466422,  -1.09922195,   2.18092847,
              4.88676171,   

In [17]:
# Reformat
from det3d.datasets.pipelines.formating import Reformat
a = Reformat()
res,frame_data = a(res, frame_data)

No Tensorflow


In [18]:
res

{'metadata': {'image_prefix': PosixPath('data/etri3Dobj_infra_edge'),
  'num_point_features': 3},
 'points': array([[ -5.73508612,   4.17013045,  -1.90411989],
        [ -6.81193661,   4.66280996,  -1.90944888],
        [  8.49280075, -50.27583465,  10.76253949],
        ...,
        [  5.97048998,  -5.29747305,  -0.14517664],
        [ 13.05656451,  -1.48573312,  -0.69696109],
        [  0.89737355,  -1.33217416,  -0.65877836]]),
 'voxels': array([[[ -5.73508612,   4.17013045,  -1.90411989],
         [ -5.79017402,   4.09995489,  -1.90515261],
         [ -5.74815793,   4.1520937 ,  -1.90411989],
         ...,
         [  0.        ,   0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        ]],
 
        [[ -6.81193661,   4.66280996,  -1.90944888],
         [ -6.82334213,   4.63920648,  -1.90855168],
         [ -6.83466754,   4.61557783,  -1.90765436],
         ...,
         [  0.        ,   0.        ,   0.     

### Voxelization 분석

In [15]:
from det3d.core.input.voxel_generator import VoxelGenerator
vxg = cfg.voxel_generator
range = vxg.range
voxel_size = vxg.voxel_size
max_points_in_voxel = vxg.max_points_in_voxel
max_voxel_num = [vxg.max_voxel_num, vxg.max_voxel_num] if isinstance(vxg.max_voxel_num, int) else vxg.max_voxel_num

double_flip = vxg.get('double_flip', False)

voxel_generator = VoxelGenerator(
    voxel_size=voxel_size,
    point_cloud_range=range,
    max_num_points=max_points_in_voxel,
    max_voxels=max_voxel_num[0])

voxel_size = voxel_generator.voxel_size
pc_range = voxel_generator.point_cloud_range
grid_size = voxel_generator.grid_size

if res["mode"] == "train":
    gt_dict = res["lidar"]["annotations"]
    bv_range = pc_range[[0, 1, 3, 4]]
    mask = prep.filter_gt_box_outside_range(gt_dict["gt_boxes"], bv_range)
    _dict_select(gt_dict, mask)

    res["lidar"]["annotations"] = gt_dict
    max_voxels = max_voxel_num[0]
else:
    max_voxels = max_voxel_num[1]

voxels, coordinates, num_points = voxel_generator.generate(
    res["lidar"]["points"], max_voxels=max_voxels 
)
num_voxels = np.array([voxels.shape[0]], dtype=np.int64)

res["lidar"]["voxels"] = dict(
    voxels=voxels,
    coordinates=coordinates,
    num_points=num_points,
    num_voxels=num_voxels,
    shape=grid_size,
    range=pc_range,
    size=voxel_size
)

double_flip = double_flip and (res["mode"] != 'train')

if double_flip:
    flip_voxels, flip_coordinates, flip_num_points = voxel_generator.generate(
        res["lidar"]["yflip_points"]
    )
    flip_num_voxels = np.array([flip_voxels.shape[0]], dtype=np.int64)

    res["lidar"]["yflip_voxels"] = dict(
        voxels=flip_voxels,
        coordinates=flip_coordinates,
        num_points=flip_num_points,
        num_voxels=flip_num_voxels,
        shape=grid_size,
        range=pc_range,
        size=voxel_size
    )

    flip_voxels, flip_coordinates, flip_num_points = voxel_generator.generate(
        res["lidar"]["xflip_points"]
    )
    flip_num_voxels = np.array([flip_voxels.shape[0]], dtype=np.int64)

    res["lidar"]["xflip_voxels"] = dict(
        voxels=flip_voxels,
        coordinates=flip_coordinates,
        num_points=flip_num_points,
        num_voxels=flip_num_voxels,
        shape=grid_size,
        range=pc_range,
        size=voxel_size
    )

    flip_voxels, flip_coordinates, flip_num_points = voxel_generator.generate(
        res["lidar"]["double_flip_points"]
    )
    flip_num_voxels = np.array([flip_voxels.shape[0]], dtype=np.int64)

    res["lidar"]["double_flip_voxels"] = dict(
        voxels=flip_voxels,
        coordinates=flip_coordinates,
        num_points=flip_num_points,
        num_voxels=flip_num_voxels,
        shape=grid_size,
        range=pc_range,
        size=voxel_size
    )            


In [ ]:
# from det3d.datasets.pipelines.preprocess import AssignLabel
# a = AssignLabel(cfg=cfg.assigner)
# res,frame_data = a(res, frame_data)

In [ ]:
res

### PreProcess 분석

In [ ]:
from det3d.torchie import Config
args.config = "configs/etriInfra/pp/etriInfra_centerpoint_pp_02voxel_two_pfn_10sweep.py"
cfg = Config.fromfile(args.config)

In [ ]:
for k in cfg.keys():
    print(k)

In [ ]:
#Preprocess
tasks = [
    dict(num_class=2, class_names=["car", "personal_mobility"]),
    dict(num_class=2, class_names=["truck", "construction_vehicle"]),
    dict(num_class=1, class_names=["bus"]),
    dict(num_class=1, class_names=["ground_animal"]),
    dict(num_class=2, class_names=["motorcycle", "bicycle"]),
    dict(num_class=1, class_names=["pedestrian"]),
]

class_names = list(itertools.chain(*[t["class_names"] for t in tasks]))
train_preprocessor = dict(
    mode="train",
    shuffle_points=True,
    global_rot_noise=[-0.3925, 0.3925],
    global_scale_noise=[0.95, 1.05],
    db_sampler=cfg.db_sampler,
    class_names=class_names,
    min_points_in_gt = 5,
)
shuffle_points = train_preprocessor["shuffle_points"]
min_points_in_gt = train_preprocessor.get("min_points_in_gt", -1)
mode = train_preprocessor["mode"]
print(f"shuffle_points: {shuffle_points}")
print(f"min_points_in_gt: {min_points_in_gt}")
print(f"mode: {mode}")

In [ ]:
from det3d.builder import build_dbsampler
if mode == "train":
    global_rotation_noise = train_preprocessor["global_rot_noise"]
    global_scaling_noise = train_preprocessor["global_scale_noise"]
    global_translate_std = train_preprocessor.get('global_translate_std', 0)
    class_names = train_preprocessor["class_names"]
print(f"global_rotation_noise: {global_rotation_noise}")
print(f"global_scaling_noise: {global_scaling_noise}")
print(f"global_translate_std: {global_translate_std}")
print(f"class_names: {class_names}")
if mode == "train":
    if train_preprocessor["db_sampler"] != None:
        db_sampler = build_dbsampler(train_preprocessor["db_sampler"])
    else:
        db_sampler = None 
    
no_augmentation = cfg.get('no_augmentation', False)

In [ ]:
info_path = "data/etri3Dobj_infra_edge/db_meta_withoutvelo.pkl"
with open(info_path, "rb") as f:
    db_infos = pickle.load(f)
db_infos

In [ ]:
import numpy as np
from det3d.core.bbox import box_np_ops
def _dict_select(dict_, inds):
    for k, v in dict_.items():
        if isinstance(v, dict):
            _dict_select(v, inds)
        else:
            dict_[k] = v[inds]
def drop_arrays_by_name(gt_names, used_classes):
    inds = [i for i, x in enumerate(gt_names) if x not in used_classes]
    inds = np.array(inds, dtype=np.int64)
    return inds
res["mode"] = mode
points = res["lidar"]["points"]
if mode == "train":
    anno_dict = res["lidar"]["annotations"]
    
    gt_dict = {
        "gt_boxes": anno_dict["boxes"],
        "gt_names": np.array(anno_dict["names"]).reshape(-1),
    }
    
if mode == "train" and not no_augmentation:
    selected = drop_arrays_by_name(
                gt_dict["gt_names"], ["DontCare", "ignore", "UNKNOWN"]
            )
    _dict_select(gt_dict, selected)
    
    if min_points_in_gt > 0:
        point_counts = box_np_ops.points_count_rbbox(
            points, gt_dict["gt_boxes"]
        )
        mask = point_counts >= min_points_in_gt
        _dict_select(gt_dict, mask)
    
    gt_boxes_mask = np.array(
                [n in class_names for n in gt_dict["gt_names"]], dtype=np.bool_
            )
gt_dict

In [ ]:
if db_sampler:
    sampled_dict = db_sampler.sample_all(
        "",
        gt_dict["gt_boxes"],
        gt_dict["gt_names"],
        res["metadata"]["num_point_features"],
        False,
        gt_group_ids=None,
        calib=None,
        road_planes=None
    )

    if sampled_dict is not None:
        sampled_gt_names = sampled_dict["gt_names"]
        sampled_gt_boxes = sampled_dict["gt_boxes"]
        sampled_points = sampled_dict["points"]
        sampled_gt_masks = sampled_dict["gt_masks"]
        gt_dict["gt_names"] = np.concatenate(
            [gt_dict["gt_names"], sampled_gt_names], axis=0
        )
        gt_dict["gt_boxes"] = np.concatenate(
            [gt_dict["gt_boxes"], sampled_gt_boxes]
        )
        gt_boxes_mask = np.concatenate(
            [gt_boxes_mask, sampled_gt_masks], axis=0
        )


        points = np.concatenate([sampled_points, points], axis=0)

_dict_select(gt_dict, gt_boxes_mask)

In [ ]:
from det3d.core.sampler import preprocess as prep
gt_classes = np.array(
    [class_names.index(n) + 1 for n in gt_dict["gt_names"]],
    dtype=np.int32,
)
gt_dict["gt_classes"] = gt_classes

gt_dict["gt_boxes"], points = prep.random_flip_both(gt_dict["gt_boxes"], points)

gt_dict["gt_boxes"], points = prep.global_rotation(
    gt_dict["gt_boxes"], points, rotation=global_rotation_noise
)
gt_dict["gt_boxes"], points = prep.global_scaling_v2(
    gt_dict["gt_boxes"], points, *global_scaling_noise
)
gt_dict["gt_boxes"], points = prep.global_translate_(
    gt_dict["gt_boxes"], points, noise_translate_std=global_translate_std
)

In [ ]:
gt_dict

In [ ]:
shuffle_points

In [ ]:
if shuffle_points:
    np.random.shuffle(points)

res["lidar"]["points"] = points

if mode == "train":
    res["lidar"]["annotations"] = gt_dict

In [ ]:
res